# Test

In [1]:
import molsysmt as msm

In [2]:
import openmm as mm
from openmm import app
from openmm import unit

In [3]:
molsys = msm.convert('4POC')

In [4]:
molsys = msm.extract(molsys, selection='molecule_type=="protein"')

In [5]:
molsys = msm.build.add_missing_hydrogens(molsys)

In [6]:
molsys = msm.build.solvate(molsys)

In [7]:
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3p.xml')

In [8]:
topology = msm.convert(molsys, 'openmm.Topology')

In [9]:
system = forcefield.createSystem(topology, nonbondedMethod=app.PME,
        nonbondedCutoff=1.2*unit.nanometer, constraints=app.HBonds)

In [10]:
integrator = mm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picosecond, 2.0*unit.femtoseconds)

In [11]:
platform = mm.Platform.getPlatformByName("CUDA")

In [12]:
simulation = app.Simulation(topology, system, integrator, platform)

In [13]:
coordinates = msm.get(molsys, coordinates=True)
positions = msm.pyunitwizard.convert(coordinates[0], to_form='openmm.unit')

In [14]:
simulation.context.setPositions(positions)
simulation.minimizeEnergy()

In [15]:
from molsysmt.thirds.openmm.reporters import MolSysMTTrajectoryDictReporter
from molsysmt.thirds.openmm.reporters import MSMH5Reporter

In [16]:
dict_reporter = MolSysMTTrajectoryDictReporter(10)
simulation.reporters.append(dict_reporter)

In [17]:
msmh5_reporter = MSMH5Reporter('test.msmh5', 10, selection='all', steps=100,
            topology=True, time=True, box=True, coordinates=True, velocities=False,
            potentialEnergy=True, kineticEnergy=True, temperature=True,
            includeInitialContext=True, constantReportInterval=True,
            constantStepSize=True, constantBox=True,
            compression='gzip', compression_opts=4,
            int_precision='single', float_precision='single',
            syntax='MolSysMT')
simulation.reporters.append(msmh5_reporter)

TypeError: MSMH5FileHandler.__init__() got an unexpected keyword argument 'length_unit'

In [ ]:
simulation.step(100)

In [ ]:
traj = dict_reporter.finalize()
msmh5_reporter.close()